## Importar librerías

In [158]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
from onnxmltools.convert import convert_lightgbm
from onnxmltools.convert.lightgbm.operator_converters.LightGbm import convert_lightgbm
import warnings
warnings.filterwarnings('ignore')

In [152]:
# Cargar
tester_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Tester/6C3C6A11D1C3791DD4DBF45421BF8028/Agent-127.0.0.1-3000/MQL5/Files"
df_buy = pd.read_csv(os.path.join(tester_files_folder, "buy_dataset.csv"))
df_sell = pd.read_csv(os.path.join(tester_files_folder, "sell_dataset.csv"))
print(f"Buy -> Trades: {df_buy.shape[0]} Features: {df_buy.shape[1]-1}")
print(f"Sell -> Trades: {df_sell.shape[0]} Features: {df_sell.shape[1]-1}")

Buy -> Trades: 1242 Features: 60
Sell -> Trades: 650 Features: 60


In [153]:
# Dividir 
X_buy = df_buy.drop(columns='target').values
y_buy = df_buy['target'].values
X_buy_train, X_buy_test, y_buy_train, y_buy_test = train_test_split(X_buy, y_buy, test_size=0.2, random_state=42)
X_sell = df_sell.drop(columns='target').values
y_sell = df_sell['target'].values
X_sell_train, X_sell_test, y_sell_train, y_sell_test = train_test_split(X_sell, y_sell, test_size=0.2, random_state=42)

In [154]:
param_grid = {
    'feature_selection__threshold': ['mean', 'median', 0.01, 0.02],
    'xgb__n_estimators': [100, 200, 300, 500],
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'xgb__subsample': [0.6, 0.8, 1.0],
    'xgb__colsample_bytree': [0.6, 0.8, 1.0],
    'xgb__gamma': [0, 0.1, 0.2],
    #'scaler__with_mean': [True, False],  # Hiperparámetro del escalador
    #'scaler__with_std': [True, False]    # Hiperparámetro del escalador
}


In [155]:
# Crear pipe y entrenar
pipe_buy = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', SelectFromModel(xgb.XGBClassifier(
        use_label_encoder=False, eval_metric='mlogloss'
    ))),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_buy = GridSearchCV(
    estimator=pipe_buy,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_buy.fit(X_buy_train, y_buy_train)
model_buy = grid_search_buy.best_estimator_
pipe_sell = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', SelectFromModel(xgb.XGBClassifier(
        use_label_encoder=False, eval_metric='mlogloss'
    ))),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_sell = GridSearchCV(
    estimator=pipe_sell,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_sell.fit(X_sell_train, y_sell_train)
model_sell = grid_search_sell.best_estimator_

Fitting 5 folds for each of 64 candidates, totalling 320 fits
Fitting 5 folds for each of 64 candidates, totalling 320 fits


In [156]:
update_registered_converter(
    xgb.XGBClassifier,
    "XGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={'nocl': [False], 'zipmap': [True, False, 'columns']}
)

In [157]:
model_buy_onnx = convert_sklearn(
    model_buy,
    'pipeline_buy_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
model_sell_onnx = convert_sklearn(
    model_sell,
    'pipeline_sell_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
mql5_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files"
with open(os.path.join(mql5_files_folder, "xgboost_buy.onnx"), 'wb') as f:
    f.write(model_buy_onnx.SerializeToString())
with open(os.path.join(mql5_files_folder, "xgboost_sell.onnx"), 'wb') as f:
    f.write(model_sell_onnx.SerializeToString())